**Target:**
*  Get the set-up right
*  Set Transforms
*  Set Data Loader
*  Set Basic Working Code
*  Set Basic Training  & Test Loop

**Results:**
*  Parameters: 16,474
*  Best Training Accuracy: 99.09
*  Best Test Accuracy: 98.55

**Analysis:**
* Huge jump of training accuracy from EPOCH 0 to 1
*  For initial few epochs, training accuracy was lower than test accuracy.
*  There is no overfitting.



In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = nn.Sequential(nn.Conv2d(1, 8, 3, padding=0),nn.ReLU(inplace=True)) #input -28x28x1 OUtput - 26x26x8 
        self.conv2 = nn.Sequential(nn.Conv2d(8, 16, 3, padding=0),nn.ReLU(inplace=True))   #input -26x26x8 OUtput - 24x24x16 
        self.conv3 = nn.Sequential(nn.Conv2d(16, 32, 3, padding=0),nn.ReLU(inplace=True))  #input -24x24x16 OUtput - 22x22x32 


        self.conv4 = nn.Sequential(nn.Conv2d(32, 8, 1, padding=0),nn.ReLU(inplace=True)) #input -22x22x32 OUtput - 22x22x8 
        self.pool1 = nn.Sequential(nn.MaxPool2d(kernel_size=2, stride=2))  #input - 22x22x8 OUtput - 11x11x8 

        self.conv5 = nn.Sequential(nn.Conv2d(8, 16, 3, padding=0),nn.ReLU(inplace=True))  #input -11x11x8 OUtput - 9x19x16

        self.conv6 = nn.Sequential(nn.Conv2d(16, 8, 1, padding=0),nn.ReLU(inplace=True)) #input -9x9x16 OUtput - 9x9x8 
        
        self.conv7 = nn.Sequential(nn.Conv2d(8, 16, 3, padding=0),nn.ReLU(inplace=True))   #input -9x9x8 OUtput - 7x7x16 
        
        self.conv8 = nn.Sequential(nn.Conv2d(16, 10, 7))   #input -7x7x16 OUtput - 1x1x10 

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.pool1(x)
        x = self.conv5(x)
        x = self.conv6(x)
        x = self.conv7(x)
        x = self.conv8(x)        
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [5]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              80
              ReLU-2            [-1, 8, 26, 26]               0
            Conv2d-3           [-1, 16, 24, 24]           1,168
              ReLU-4           [-1, 16, 24, 24]               0
            Conv2d-5           [-1, 32, 22, 22]           4,640
              ReLU-6           [-1, 32, 22, 22]               0
            Conv2d-7            [-1, 8, 22, 22]             264
              ReLU-8            [-1, 8, 22, 22]               0
         MaxPool2d-9            [-1, 8, 11, 11]               0
           Conv2d-10             [-1, 16, 9, 9]           1,168
             ReLU-11             [-1, 16, 9, 9]               0
           Conv2d-12              [-1, 8, 9, 9]             136
             ReLU-13              [-1, 8, 9, 9]               0
           Conv2d-14             [-1, 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:32: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [6]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...
Done!


In [0]:
from tqdm import tqdm

train_losses = []
test_losses = []
train_acc = []
test_acc = []

def train(model, device, train_loader, optimizer, epoch):
  model.train()
  pbar = tqdm(train_loader)
  correct = 0
  processed = 0
  for batch_idx, (data, target) in enumerate(pbar):
    # get samples
    data, target = data.to(device), target.to(device)

    # Init
    optimizer.zero_grad()
    # In PyTorch, we need to set the gradients to zero before starting to do backpropragation because PyTorch accumulates the gradients on subsequent backward passes. 
    # Because of this, when you start your training loop, ideally you should zero out the gradients so that you do the parameter update correctly.

    # Predict
    y_pred = model(data)

    # Calculate loss
    loss = F.nll_loss(y_pred, target)
    train_losses.append(loss)

    # Backpropagation
    loss.backward()
    optimizer.step()

    # Update pbar-tqdm
    
    pred = y_pred.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
    correct += pred.eq(target.view_as(pred)).sum().item()
    processed += len(data)

    pbar.set_description(desc= f'Loss={loss.item()} Batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')
    train_acc.append(100*correct/processed)

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
    test_acc.append(100. * correct / len(test_loader.dataset))

In [8]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
EPOCHS = 15
for epoch in range(EPOCHS):
    print("EPOCH : ",epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]

EPOCH :  0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:32: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
Loss=0.43227314949035645 Batch_id=468 Accuracy=33.62: 100%|██████████| 469/469 [00:11<00:00, 42.25it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.3252, Accuracy: 8988/10000 (89.88%)

EPOCH :  1


Loss=0.08563558012247086 Batch_id=468 Accuracy=94.74: 100%|██████████| 469/469 [00:10<00:00, 43.15it/s]
  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0484, Accuracy: 9838/10000 (98.38%)

EPOCH :  13


Loss=0.05629928037524223 Batch_id=468 Accuracy=99.02: 100%|██████████| 469/469 [00:10<00:00, 43.14it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0488, Accuracy: 9855/10000 (98.55%)

EPOCH :  14


Loss=0.006349270697683096 Batch_id=468 Accuracy=99.09: 100%|██████████| 469/469 [00:10<00:00, 43.50it/s]



Test set: Average loss: 0.0522, Accuracy: 9843/10000 (98.43%)

